In [1]:
import pickle
from collections import Counter
from functools import reduce
from math import pow

In [2]:
with open('noise_models/error_rates.pkl' , 'rb') as f:
    error_rates = pickle.load(f)

In [3]:
def add_circuit_reliability(N,T,system):
    with open('circuits/{}_{}_{}.pkl'.format(system,N,str(T).replace('.','-')), 'rb') as f:
        circuit_params = pickle.load(f)

    circuit_reliabilities = []

    for circuit in circuit_params['circuit']:
        rzx_instructions_counts = Counter(list(map(lambda x: (x.qubits[0]._index, x.qubits[1]._index), circuit.get_instructions('rzx'))))
        rx_instructions_counts = Counter(list(map(lambda x: x.qubits[0]._index, circuit.get_instructions('rx'))))
        rz_instructions_counts = Counter(list(map(lambda x: x.qubits[0]._index, circuit.get_instructions('rz'))))
        sx_instructions_counts = Counter(list(map(lambda x: x.qubits[0]._index, circuit.get_instructions('sx'))))

        rzx_reliability = reduce(lambda x,y: x*y, list(map(lambda x: pow(1 - error_rates[system]['rzx'][x[0]], x[1]),rzx_instructions_counts.items())))
        rx_reliability = reduce(lambda x,y: x*y, list(map(lambda x: pow(1 - error_rates[system]['rx'][x[0]], x[1]),rx_instructions_counts.items())))
        sx_reliability = reduce(lambda x,y: x*y, list(map(lambda x: pow(1 - error_rates[system]['sx'][x[0]], x[1]),sx_instructions_counts.items())))
        rz_reliability = reduce(lambda x,y: x*y, list(map(lambda x: pow(1 - error_rates[system]['rz'][x[0]], x[1]),rz_instructions_counts.items())))

        circuit_reliability = rzx_reliability * rx_reliability * sx_reliability * rz_reliability
        circuit_reliabilities.append(circuit_reliability)
    
    circuit_params['circuit_reliabilities'] = circuit_reliabilities
    print("N = {}, T = {}, system = {}".format(N,T,system))
    print(circuit_reliabilities)

    with open('circuits/{}_{}_{}.pkl'.format(system,N,str(T).replace('.','-')), 'wb') as f:
        pickle.dump(circuit_params, f)

In [4]:
for N in range(4,11):
    for T in [0.25,0.5,0.75,1.0,1.5,2,2.5,3,5,10]:
        for system in ['ibm_brisbane','ibm_sherbrooke','ibm_kyoto','ibm_nazca']:
            add_circuit_reliability(N,T,system)

N = 4, T = 0.25, system = ibm_brisbane
[0.845740815951508, 0.6100744093010173, 0.4400766498010961, 0.3174489123745543, 0.2289914995793]
N = 4, T = 0.25, system = ibm_sherbrooke
[0.8814928297192339, 0.686053553503251, 0.5339458954241886, 0.4155626303289565, 0.3234265891092339]
N = 4, T = 0.25, system = ibm_kyoto
[0.7476705639848552, 0.4203927026599226, 0.23637419067001536, 0.13290610817310772, 0.07472911293594345]
N = 4, T = 0.25, system = ibm_nazca
[0.848895888866562, 0.6182585015648159, 0.4502832205578703, 0.3279453144644088, 0.23884551848482186]
N = 4, T = 0.5, system = ibm_brisbane
[0.845740815951508, 0.6100744093010173, 0.4400766498010961, 0.3174489123745543, 0.2289914995793]
N = 4, T = 0.5, system = ibm_sherbrooke
[0.8814928297192339, 0.686053553503251, 0.5339458954241886, 0.4155626303289565, 0.3234265891092339]
N = 4, T = 0.5, system = ibm_kyoto
[0.7476705639848552, 0.4203927026599226, 0.23637419067001536, 0.13290610817310772, 0.07472911293594345]
N = 4, T = 0.5, system = ibm_naz